In [17]:
import time

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

import numpy as np
from torchvision import datasets
from load_data import CustomDataset, CustomTestDataset
import pandas as pd
import cv2
import os

from typing import Callable
import torch
import torch.optim
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, Resize, Normalize, CenterCrop

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
batch_size = 256
num_epochs = 3

optimizer_factory: Callable[
    [nn.Module], torch.optim.Optimizer
] = lambda model: torch.optim.Adam(model.parameters(), lr=0.001)

resize = (224,224)

transforms = Compose(
    [   
        ToTensor(),
        # CenterCrop((480, 640)),
        Resize(resize),
        # Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616)),
        # Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        # RandomRotation(1),
        # ColorJitter(brightness = 0.1, contrast = 0.1, saturation = 0.1),
        # Normalize(mean=[0.485, 0.456, 0.4], std=[0.229, 0.224, 0.2])
    ]
)

In [35]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, num_channels, resize, transform = None, device = "cpu"):
        self.df = pd.read_csv(csv_path)
        self.device = device
        self.images_folder = images_folder
        self.transform = transform
        self.img_tensor, self.label_tensor = self.load_data(num_channels, resize)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        return (self.img_tensor[index], self.label_tensor[index])
    
    def load_data(self, num_channels, resize):
        # img_tensor = torch.zeros((len(self.df), num_channels, resize[0], resize[1])).to(self.device)
        # label_tensor = torch.zeros((len(self.df)))
        for i in range(len(self.df)):
            img_name = os.path.join(self.images_folder, self.df.iloc[i, 0])
            image = cv2.imread(img_name)
            label = torch.tensor(int(self.df.iloc[i, -1])).to(self.device)
            if self.transform:
                image = self.transform(image).to(self.device)
            if i==0:
                img_tensor = image
                label_tensor = [label]
                # print(label_tensor)
            else:
                img_tensor = torch.stack((img_tensor, torch.unsqueeze(image, 0)), 0)
                label_tensor.append([label])
            print(img_tensor.shape)    
        return img_tensor, torch.tensor(label_tensor).to(self.device)
    
    def load_data1(self):
        f = open("./train.csv", "r")
        l = f.readlines()
        # img_path_list = glob(data_path)
        for i in range(1, len(l)):
            line = l[i].split(",")
            img_path = line[0]
            lab = torch.tensor(int(line[-1].strip()))
            # tranform the image to same size
            image = cv2.imread(os.path.join(self.images_folder, img_path))
            if self.transform:
                image = self.transform(image)
                
            if i == 1:
                img_tensor = image
                label = lab
            else:
                img_tensor = torch.hstack((img_tensor, image))
                label = torch.hstack((label, lab))
        return img_tensor, label
    
class CustomTestDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, num_channels, resize, transform = None, device = "cpu"):
        self.df = pd.read_csv(csv_path)
        self.device = device
        self.images_folder = images_folder
        self.transform = transform
        self.img_tensor = self.load_data(num_channels, resize)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        return self.img_tensor[index]
    
    def load_data(self, num_channels, resize):
        img_tensor = torch.zeros((len(self.df), num_channels, resize[0], resize[1])).to(self.device)
        for i in range(len(self.df)):
            img_name = os.path.join(self.images_folder, self.df.iloc[i, 0])
            image = cv2.imread(img_name)
            if self.transform:
                image = self.transform(image).to(self.device)
            img_tensor[i,:,:,:] = image
        return img_tensor

In [36]:
# Create the data loaders:
dataset = CustomDataset("./train.csv", "./images", 3, resize, transforms, "cuda")
test_loader = DataLoader(CustomTestDataset("./test.csv", "./images", 3, resize, transforms), 
    batch_size=batch_size, shuffle=True)
train, val = torch.utils.data.random_split(dataset, [0.7, 0.3])
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)

torch.Size([3, 224, 224])
torch.Size([2, 3, 224, 224])


/home/singroa/miniconda3/envs/cis522/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


RuntimeError: stack expects each tensor to be equal size, but got [2, 3, 224, 224] at entry 0 and [3, 224, 224] at entry 1

In [ ]:
# Create the model:
model = Model(num_channels=3, num_classes=2)
# Create the optimizer:
optimizer = optimizer_factory(model)
# Create the loss function:
criterion = torch.nn.CrossEntropyLoss()
# Train the model:
tic = time.time()
num_epochs = 10
num_channels = 3

In [12]:
# Move the model to the device:
model.to(device)
train_loss_list, val_loss_list = [], []
train_acc, validation_acc = [], []
# Loop over the epochs:
for epoch in range(num_epochs):
    train, val = torch.utils.data.random_split(dataset, [0.7, 0.3])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
    print(dataset.img_tensor.shape)
    print(dataset.label_tensor.shape)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
    device = "cuda"
    model.to(device)
    train_loss = 0.0
    # Set the model to training mode:
    model.train()

    # Loop over the training data:
    for x, y in tqdm(train_loader):
        # Move the data to the device:
        a = x
    #     x, y = x.to(device), y.to(device)
    #     # Zero the gradients:
    #     optimizer.zero_grad()
    #     # Forward pass:
    #     y_hat = model(x)
    #     # Compute the loss:
    #     loss = criterion(y_hat, y)
    #     # Backward pass:
    #     loss.backward()
    #     # Update the parameters:
    #     optimizer.step()
    #     # Calculate Loss
    #     train_loss += loss.item()
    # train_loss_list.append(train_loss)

    # device = "cpu"
    # model.to(device)
    # valid_loss = 0.0
    # # Set the model to evaluation mode:
    # model.eval()
    # # Initialize the number of correct predictions:
    # num_correct = 0
    # pred = np.array([])
    # # Loop over the data:
    # for x, y in tqdm(val_loader):
    #     # Move the data to the device:
    #     x, y = x.to(device), y.to(device)
    #     # Forward pass:
    #     y_hat = model(x)
    #     # Compute the predictions:
    #     predictions = torch.argmax(y_hat, dim=1)
    #     pred = np.hstack((pred, predictions.cpu()))
    #     loss = criterion(y_hat,y)
    #     valid_loss += loss.item()
    #     num_correct += (predictions == y).float().sum().item()
    # val_loss_list.append(valid_loss)
    # print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(val_loader)}')
    # print("acc : ", num_correct)
    
        

torch.Size([3, 2593920, 32])
torch.Size([81060])


  0%|          | 0/222 [00:00<?, ?it/s]


IndexError: index 16972 is out of bounds for dimension 0 with size 3

In [ ]:
pred = np.hstack((pred, predictions.cpu()))

In [ ]:
import matplotlib.pyplot as plt
epochs = np.arange(num_epochs)
# plot lines
plt.plot(epochs, train_loss_list, label = "line 1")
plt.plot(epochs, val_loss_list, label = "line 2")
plt.legend()
plt.show()

In [ ]:
toc = time.time()
print(
    f"Training time: {toc - tic:.2f} seconds"
)
print(
    f"Training time: {(toc - tic)/60:.2f} mins"
)

In [ ]:
# Set the model to evaluation mode:
model.eval()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = "cpu"
# Compute the accuracy on the test data:
model.to(device)
# Set the model to evaluation mode:
model.eval()
# Initialize the number of correct predictions:
num_correct = 0
pred = np.array([])
# Loop over the data:
for x in tqdm(test_loader):
    # Move the data to the device:
    x = x.to(device)
    # Forward pass:
    y_hat = model(x)
    # Compute the predictions:
    predictions = torch.argmax(y_hat, dim=1)
    pred = np.hstack((pred, predictions))

test_df = pd.read_csv("./test.csv")
img_path = test_df["img_path"]
df = pd.DataFrame(pred)
df.insert(0, "img_path", img_path, True)
df.to_csv("submission",index=False)

In [ ]:
pred = pred.astype(int)
test_df = pd.read_csv("./test.csv")
img_path = test_df["img_path"]
df = pd.DataFrame()
df["has_under_extrusion"] = pred
df.insert(0, "img_path", img_path, True)
df.to_csv("submission.csv",index=False)